<a href="https://colab.research.google.com/github/benbaz-2/comp551/blob/main/CNN_task2.6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
!pip install medmnist

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from medmnist import OrganAMNIST
from tqdm import tqdm

In [3]:
import os
import numpy as np
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from matplotlib import pyplot
import matplotlib.image as mpimg

import torch
import torch.nn as nn
from torchvision.datasets import MNIST, FashionMNIST
from torchvision.transforms import ToTensor, Compose, Normalize, RandomHorizontalFlip, RandomRotation

## Define transforms

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

## Load data

In [5]:
train_dataset = OrganAMNIST(split='train', download=True, transform=transform)
test_dataset = OrganAMNIST(split='test', download=True, transform=transform)
val_dataset = OrganAMNIST(split='val', download=True, transform=transform)

Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz


In [6]:
# Use dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Model implementation

In [7]:
num_filters = 8
filter_size = 3
pool_size   = 2
num_classes = 10
batch_size  = 64
strides     = 1
padding     = 1  # Padding of 1 with filter size of 3 output the same dimension
dropout_rate = 0.25

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential()
        self.model.append(nn.Conv2d(1, num_filters, filter_size, strides, padding))

        self.model.append(nn.MaxPool2d(pool_size))

        self.model.append(nn.Conv2d(num_filters,  num_filters ** 2, filter_size, strides, padding))
        self.model.append(nn.ReLU())
        self.model.append(nn.MaxPool2d(pool_size))

        self.model.append(nn.Dropout(dropout_rate)) # randomly sets input units to 0 with a frequency of rate at each step during training time, to avoid overfitting
        self.model.append(nn.Flatten()) # Flatten serves as a connection between the convolution and dense layers
        self.model.append(nn.Linear(3136, 1000)) # hidden layer
        self.model.append(nn.ReLU())
        self.model.append(nn.Dropout(dropout_rate))
        self.model.append(nn.Linear(1000, num_classes))
        #self.model.append(nn.Softmax(dim=1))

    def forward(self, x):
        return self.model(x)


## Model Training

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model = CNN()
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 10

In [9]:
def compute_accuracy(model = model, data_loader = train_loader, device=device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)  # Get logits
            _, predicted = torch.max(outputs.data, 1)  # Get predicted class indices

            total += labels.size(0)  # Total number of labels
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = correct / total * 100  # Convert to percentage
    return accuracy

In [11]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
      #labels = labels.squeeze()
      images, labels = images.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels.squeeze().long())
      loss.backward()
      optimizer.step()
      running_loss += loss.item()


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
for images, labels in train_loader:
        print(images.shape, labels.shape)

array([ 6,  8,  5,  6,  9,  7,  6,  9,  4,  7,  7,  9,  5,  0, 10,  7,  7,
       10, 10,  7,  3,  7,  8,  0,  7,  2,  0,  9,  8, 10,  6,  5,  7,  4,
        5,  6,  7,  7,  6, 10,  7,  1,  6, 10,  2,  0, 10,  8,  7,  0,  0,
        6,  7, 10,  5,  4,  6,  4, 10,  5,  4,  8,  6,  5,  3,  8,  8,  2,
        9,  0,  9,  6,  6,  9,  8,  6,  4,  1,  9,  3,  6,  2,  4,  8,  6,
        5,  4,  6,  8,  0,  5, 10,  9,  8,  4,  3,  5,  4, 10,  5, 10,  8,
        7,  6,  4, 10,  1,  5,  9,  7,  8], dtype=uint8)